### Exercice 1: Initialisation et Analyse de Forme 

In [ ]:
import pandas as pd

# Setup des chemins
FILE_PATH = '../data/raw/dataset.xlsx'

In [ ]:

try:
    # Khassk tkun installiti openpyxl qbl (pip install openpyxl)
    data = pd.read_excel(FILE_PATH)
    print("✅ Fichier chargé mzyan!")
    
    # Deep Copy bash n7afdo 3la l-original
    df = data.copy()
    
except FileNotFoundError:
    print("❌ Erreur: Fichier malqinahch. Vérifier l-path a sat.")
except ImportError:
    print("❌ Erreur: Khassk t-installé openpyxl. Dir: pip install openpyxl")

In [ ]:
print("-" * 30)
print("1. Les 5 premières lignes (Head):")
display(df.head()) 

print("\n2. Dimensions du dataset (Shape):")
print(f"Lignes: {df.shape[0]}, Colonnes: {df.shape[1]}") 

print("\n3. Types de données (Dtypes):")
print(df.dtypes.value_counts())

## Exercice 2: Gestion des valeurs manquantes (NaN) 

##### QST[1]

In [ ]:
# Importation des bibliotheques de visualisation (si pas encore fait)
import seaborn as sns
import matplotlib.pyplot as plt

# Configuration de la taille de la figure
plt.figure(figsize=(12, 6))

# Affichage de la Heatmap
# df.isna() retourne True pour les valeurs manquantes (NaN)
# cbar=False enleve la barre de couleur pour plus de clarte
# yticklabels=False cache les index des lignes (trop nombreux a afficher)
sns.heatmap(df.isna(), cbar=False, yticklabels=False, cmap='viridis')

# Ajout du titre
plt.title("Visualisation des valeurs manquantes (NaN) dans le dataset")

# Affichage du graphique
plt.show()

##### QST[2]

In [ ]:
# 2. Calcul du pourcentage de valeurs manquantes (NaN) par colonne

# On compte le nombre de NaN dans chaque colonne avec isna().sum()
# On divise par le nombre total de lignes (df.shape[0]) pour avoir la proportion
# On multiplie par 100 pour avoir un pourcentage
nan_percentage = (df.isna().sum() / df.shape[0]) * 100

# On trie les résultats par ordre décroissant (du plus grand au plus petit)
# Cela permet de voir immédiatement les colonnes les plus vides
nan_percentage = nan_percentage.sort_values(ascending=False)

# Affichage des 20 premières colonnes (les plus vides)
print("Top 20 des colonnes avec le plus de valeurs manquantes :")
print(nan_percentage.head(20))
print(df.shape[1])

##### QST[3]

In [ ]:
# 3. Nettoyage des donnees

# Etape 1 : Identifier les colonnes avec plus de 90% de NaN
# On utilise la variable 'nan_percentage' qu'on a calculee juste avant
cols_to_drop = nan_percentage[nan_percentage > 90].index

# On supprime ces colonnes du dataset
# errors='ignore' permet d'eviter un crash si une colonne n'existe pas
df = df.drop(columns=cols_to_drop, errors='ignore')

print(f"{len(cols_to_drop)} colonnes ont ete supprimees car elles avaient > 90% de vide.")

# Etape 2 : Supprimer la colonne 'Patient ID'
# On verifie d'abord si elle existe encore pour eviter une erreur
if 'Patient ID' in df.columns:
    df = df.drop(columns=['Patient ID'])
    print("La colonne 'Patient ID' a ete supprimee.")
else:
    print("La colonne 'Patient ID' n'existe plus.")

##### QST[4]

In [36]:
# 4. Verification des nouvelles dimensions (Shape)
print("Nouvelles dimensions du dataset :")
print(f"Lignes : {df.shape[0]}, Colonnes : {df.shape[1]}")

Nouvelles dimensions du dataset :
Lignes : 5644, Colonnes : 38
